In [ ]:
!pip install transformers
!pip install langchain
!pip install arxiv
!pip install pymupdf

In [ ]:
from langchain.document_loaders import ArxivLoader
from transformers import pipeline

In [ ]:
def doc_loader(search_query):
  try :
    docs = ArxivLoader(query=search_query, load_max_docs=1).load()
    return docs[0].metadata, docs[0].page_content
  except Exception as e:
    print(e)

In [ ]:
def strip(content):
  content = str(content)
  #print(content)
  content = content.split("\n")
  content = " ".join(content)
  #print(content)

  return content

In [ ]:
def clip(content):
  loc_intro = content.find("Introduction")
  loc_refer = content.rfind("Reference")
  if loc_intro !=-1:
    if loc_refer !=-1:
      content = content[loc_intro:loc_refer]
    else:
      content = content[loc_intro:]
      print("Warning: Paper Doesn't have a References Title, may lead to overlap of references in summary")
  else:
    print("Warning: Paper Doesn't Have an Introduction Title, these may lead to overlap of summarization")

  return content

In [ ]:
def chunk(content):

  content = clip(content)

  sent = []
  c= 0
  k = ""
  content = content.split(". ")
  for i in range(len(content)):
    k = k + content[i] + ". "
    c = c+1
    if c == 10:
      sent.append(k)
      c = 0
      k = ""
    elif i==len(content)-1:
      sent.append(k)

  return sent

In [ ]:
def summarize(sent):
  model_str = "Falconsai/text_summarization"
  tokenizer_str = "Falconsai/text_summarization"

  summarizer = pipeline("summarization", model=model_str, tokenizer = tokenizer_str)


  summarized = ""
  for i in sent:
    s = summarizer(i, max_length=256, min_length=64, do_sample=False)
    summarized = summarized + s[0]['summary_text'] +"\n"

  return summarized

In [ ]:
def run(search_query):
  metadata, page_content = doc_loader(search_query)
  content = strip(page_content)
  sent = chunk(content)
  summarized = summarize(sent)

  print(metadata['Published'])
  print(metadata['Title'])
  print(metadata['Authors'])

  print(summarized)

In [ ]:
search_query = "Attention is all you need"
run(search_query)

2021-07-16
All the attention you need: Global-local, spatial-channel attention for image retrieval
Chull Hwan Song, Hye Joo Han, Yannis Avrithis
Instance-level image retrieval is at the core of visual rep- resentation learning . Many large-scale open datasets [3, 37, 16, 29, 53] and competitions1 have accelerated progress . Some studies focus on learning features from convolutional neural networks . Others focus on re-ranking, for instance by graph-based methods .
Each study has been limited to one or two kinds of attention only . It is the objective of our work to perform a compre-hensive study of all forms of attention above, apply them to instance-level image retrieval and provide a detailed ac- count of their interaction and impact on performance . Local at-tention is about individual locations and channels; global is about interaction between locations and between channels .
Each of the global and local attention mechanisms comprises both spatial and channel attention . Our global